## odfcontrol

`odfcontrol` contains basic methods and classes for interacting with XMM observational data files (ODF). It is **not** a new SAS task, but it can make doing basic or common analysis easier. Almost everything is contained in a single Python class called `ODFobject`.

## odfcontrol.ODFobject

`odfcontrol.ODFobject` contains the following methods:

```python
inisas(self,sas_dir,sas_ccfpath,verbosity=4,suppress_warning=1)
    
set_odfid(self)
    
sas_talk(self,verbosity=4,suppress_warning=1)
    
calibrate_odf(self,data_dir=None,level='ODF',
              sas_ccf=None,sas_odf=None,
              cifbuild_opts=None,odfingest_opts=None,
              encryption_key=None,overwrite=False,repo='esa')
                    
run_analysis(self,task,inargs,rerun=False,logFile='DEFAULT')
    
basic_setup(self,run_epproc=True,run_emproc=True,run_rgsproc=True,
            run_omichain=False,run_epchain=False,run_emchain=False,
            **kwargs)
                  
get_active_instruments(self)
    
get_event_lists(self,print_output=True)
```
<div class="alert alert-block alert-info">
    <b>Note:</b> Some of these functions are not intended to be used by the typical pySAS user. Internal pySAS use only.
</div>

The functions desinged to be used by the typical pySAS user are:

    - calibrate_odf
    - basic_setup
    - get_active_instruments
    - get_event_lists

---

When a user creates an `ODF` object using:
```python
obsid = '##########'
odf = pysas.odfcontrol.ODFobject(obsid)
```
the `ODF` object will have a few important links and information by default. These include:

    - odfid:    The ObsID number used to create the ODF object.
    - data_dir: Path to base data directory where all XMM data files 
                will be downloaded.
    - obs_dir:  Path to directory containing the data files 
                (obs_dir = data_dir/obsid).
    - odf_dir:  Path to directory with the raw observation data files 
                (odf_dir = obs_dir/ODF).
    - work_dir: The working directory when SAS tasks will be run 
                (work_dir = obs_dir/work).
    - sas_ccf:  Link to the 'ccf.cif' file, if it exists. By default the 
                'ccf.cif' file will be created in the work_dir.
    - sas_odf:  Link to the '*SUM.SAS' file, if it exists. By default the 
                '*SUM.SAS' file will be created in the work_dir.
    - If they exist, links to the basic event lists will be stored in a 
      dictionary named 'files'.

If the ODF files have previously been downloaded and are in the default data directory, then upon creating the ODF object (i.e. "odf = pysas.odfcontrol.ODFobject(obsid)") pySAS will automatically find and link all important summary and calibration files, and also all previously generated event lists made by `epproc`, `emproc`, and `rgsproc`.

In [1]:
import pysas

### odfcontrol.ODFobject

In [2]:
pysas.odfcontrol.ODFobject?

Init signature: pysas.odfcontrol.ODFobject(odfid, data_dir=None)
Docstring:     
Class for observation data files (ODF).

    An odfid (Obs ID) is required.

    data_dir is the base directory where you store all XMM data.

    Data is organized as:
        data_dir = /path/to/data/
        obs_dir = /path/to/data/odfid/
    With subdirectories and files:
            odf_dir  = /path/to/data/odfid/ODF/
            work_dir = /path/to/data/odfid/work/
            SAS_CCF  = work_dir/ccf.cif
            SAS_ODF  = work_dir/*SUM.SAS

            
    Directory links and OdfID information are stored as
    variables in the ODFobject.

        self.odfid    : The Obs ID required to make the ODFobject
        self.data_dir : Path to the data directory
        self.files    : A dictionary containing links to event lists
                        and other important files

                        Default dictionary keys are:
                            'sas_ccf'
                            'sas_

---
## Functions to be used by the end user.

### odfcontrol.ODFobject.calibrate_odf

In [3]:
pysas.odfcontrol.ODFobject.calibrate_odf?

Signature:
pysas.odfcontrol.ODFobject.calibrate_odf(
    self,
    data_dir=None,
    level='ODF',
    sas_ccf=None,
    sas_odf=None,
    cifbuild_opts=None,
    odfingest_opts=None,
    encryption_key=None,
    overwrite=False,
    recalibrate=False,
    repo='esa',
)
Docstring:
Before running this function an ODFobject must be created first. e.g.

    odf = pysas.odfcontrol.ODFobject(obsid)

This function can then be used as, odf.calibrate_odf().

The calibrate_odf function will automatically look in data_dir for the subdirectory 
data_dir/odfid. If it does not exist then it will download the data.

If it exists it will search data_dir/odfid and any subdirectories for the ccf.cif
and *SUM.SAS files. But if overwrite=True then it will remove data_dir/odfid and 
download the data.

Optionally the paths to the ccf.cif and *SUM.SAS files can be given through 
sas_ccf and sas_odf respectively.

Inputs:
    --REQUIRED--

        NONE

    --OPTIONAL--

    --data_dir:  (string/path): Path

### odfcontrol.ODFobject.basic_setup

In [4]:
pysas.odfcontrol.ODFobject.basic_setup?

Signature:
pysas.odfcontrol.ODFobject.basic_setup(
    self,
    run_epproc=True,
    run_emproc=True,
    run_rgsproc=True,
    run_omichain=False,
    run_epchain=False,
    run_emchain=False,
    **kwargs,
)
Docstring:
Function to do all basic analysis tasks. The function will:

    1. Call the function 'calibrate_odf'
        A. Download data
        B. Run 'cifbuild'
        C. Run 'odfingest'
    2. Run 'epproc' -OR- 'epchain'
    3. Run 'emproc' -OR- 'emchain'
    4. Run 'rgsproc'
    5. Run 'omichain' (not run by default)

If 'run_epchain' is set to 'True', then 'epproc' will not run.
If 'run_emchain' is set to 'True', then 'emproc' will not run.

All input arguments for 'calibrate_odf' can be passed to 'basic_setup'.

'calibrate_odf' inputs (with defaults):
       
    data_dir       = None
    level          = 'ODF'
    sas_ccf        = None
    sas_odf        = None
    cifbuild_opts  = None
    odfingest_opts = None
    encryption_key = None
    overwrite      = False
    r

### odfcontrol.ODFobject.get_active_instruments

In [5]:
pysas.odfcontrol.ODFobject.get_active_instruments?

Signature: pysas.odfcontrol.ODFobject.get_active_instruments(self)
Docstring:
Checks odf summary file for which instruments were active for that odf.

Assumes that 'sas_odf' already exists and contains the correct path.

Also assumes file name and path are stored in self.files['sas_odf'].
File:      ~/XMMGOF/xmmsas_20230412_1735/lib/python/pysas/odfcontrol/odfcontrol.py
Type:      function

### odfcontrol.ODFobject.get_event_lists

In [6]:
pysas.odfcontrol.ODFobject.get_event_lists?

Signature: pysas.odfcontrol.ODFobject.get_event_lists(self, print_output=True)
Docstring:
Checks the observation directory (obs_dir) for basic unfiltered 
event list files created by 'epproc', 'emproc', 'rgsproc', and 
'omichain'. Stores paths and file names in self.files.

'self.files' is a dictionary with the following keys:

    'PNevt_list'
    'M1evt_list'
    'M2evt_list'
    'R1evt_list'
    'R2evt_list'
    'OMevt_list'
File:      ~/XMMGOF/xmmsas_20230412_1735/lib/python/pysas/odfcontrol/odfcontrol.py
Type:      function

---
## Functions not intended for general use. Internal pySAS use only.

### odfcontrol.ODFobject.inisas

In [7]:
pysas.odfcontrol.ODFobject.inisas?

Signature:
pysas.odfcontrol.ODFobject.inisas(
    self,
    sas_dir,
    sas_ccfpath,
    verbosity=4,
    suppress_warning=1,
)
Docstring:
--Not intended to be used by the end user. Internal use only.--

Simple wrapper for 'initializesas' defined in init_sas.py.

SAS initialization should happen automatically.
File:      ~/XMMGOF/xmmsas_20230412_1735/lib/python/pysas/odfcontrol/odfcontrol.py
Type:      function

### odfcontrol.ODFobject.set_odfid

In [8]:
pysas.odfcontrol.ODFobject.set_odfid?

Signature: pysas.odfcontrol.ODFobject.set_odfid(self)
Docstring:
--Not intended to be used by the end user. Internal use only.--

Basic method for setting the environment variables for a single 
'ObsID'.

Checks if obs_dir exists in data_dir. If it exists looks for 
existing odf_dir, work_dir, SAS_CCF, and SAS_ODF. Similar to 
calibrate_odf, but will not download any data, will not 
calibrate it, or do anything other than link to files and 
directories. 
File:      ~/XMMGOF/xmmsas_20230412_1735/lib/python/pysas/odfcontrol/odfcontrol.py
Type:      function

### odfcontrol.ODFobject.sas_talk

In [9]:
pysas.odfcontrol.ODFobject.sas_talk?

Signature: pysas.odfcontrol.ODFobject.sas_talk(self, verbosity=4, suppress_warning=1)
Docstring:
Simple function to set general SAS veriables 'verbosity' 
and 'suppress_warning'.
File:      ~/XMMGOF/xmmsas_20230412_1735/lib/python/pysas/odfcontrol/odfcontrol.py
Type:      function

### odfcontrol.ODFobject.run_analysis

In [10]:
pysas.odfcontrol.ODFobject.run_analysis?

Signature:
pysas.odfcontrol.ODFobject.run_analysis(
    self,
    task,
    inargs,
    rerun=False,
    logFile='DEFAULT',
)
Docstring:
--Not intended to be used by the end user. Internal use only.--

A wrapper for the wrapper. Yes. I know.

This function is not intended to be used by the end user, but is
only called by 'basic_setup'.

This will check if output files are present for the selected SAS task.
If they are, will not rerun that SAS task unless "rerun=True".

Lists of output files are stored in the dictionary self.files{}.

SAS Tasks that it currently works for:
    --epproc
    --epchain (Warning current version of epchain fails in SAS v. 21)
    --emproc
    --emchain (Warning current version of emchain fails in SAS v. 21)
    --rgsproc

More will be added as needed.
File:      ~/XMMGOF/xmmsas_20230412_1735/lib/python/pysas/odfcontrol/odfcontrol.py
Type:      function

---
### odfcontrol.download_data

In [11]:
pysas.odfcontrol.download_data?

Signature:
pysas.odfcontrol.download_data(
    odfid,
    data_dir,
    level='ODF',
    encryption_key=None,
    repo='esa',
    logger=None,
)
Docstring:
--Not intended to be used by the end user. Internal use only.--
--Recommended to use pysas.odfcontrol.ODFobject.calibrate_odf instead.--

Downloads, or copies, data from chosen repository. 

Will silently overwrite any preexisting data files and remove any existing
pipeline products. Will create directory structure in 'data_dir' for odf.

Inputs:

        --odfid:          (string): ID of ODF in string format.

        --data_dir:  (string/path): Path to directory where the data will be 
                                    downloaded. Automatically creates directory
                                    data_dir/odfid.
                                    Default: 'pwd', returns the current directory.

        --level:          (string): Level of data products to download.
                                    Default: 'ODF'
            